<a href="https://colab.research.google.com/github/yifan-zhou19/iso/blob/master/iso_embedding_ast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from os.path import exists

if not exists('astcfg.zip'):
    !wget -O astcfg.zip https://raw.githubusercontent.com/yifan-zhou19/iso/master/AST_CFG.zip
    !unzip astcfg.zip

## **AST Embedding**

In [0]:
import os.path
import re
source_code={78:'CWE78_ast',80:'CWE80_ast',89:'CWE89_ast'}
def eachFile(filepath):
  pathDir =  os.listdir(filepath)
  files = []
  for allDir in pathDir:
    child = os.path.join('%s/%s' % (filepath, allDir))
    
    files.append(child)
  return files
root_dir = './AST_CFG/' # refer root dir
files78 = eachFile(root_dir+source_code[78])
files80 = eachFile(root_dir+source_code[80])
files89 = eachFile(root_dir+source_code[89])
files=files78[:700]+files80[:700]+files89[:700]
labels=[1]*700+[2]*700+[3]*700

In [39]:
def extract_ast_edges(file):
  results = []
  with open(file, "r") as f:
    for line in f.readlines()[2:]:
      #print(line)
      result = re.search(r'\bn(\w+)(\s*)-(\S+)(\s*)n(\w+)\b',line)
      if result != None:
         results.append((result.group(1),result.group(5)))
    return results
def extract_source_code(file):
  results = []
  with open(file, "r") as f:
    for line in f.readlines()[2:]:
      #print(line)
      result = re.search(r'\b(n\w+)(\s+\[label=")([\w:\s\S]+)\b',line)
      if result != None:
         results.append(result.group(3))
    return results
print(extract_ast_edges(files[1]))
print(extract_source_code(files[1]))

[('1', '2'), ('1', '3'), ('3', '4'), ('3', '5'), ('1', '6'), ('6', '7'), ('6', '8'), ('6', '9'), ('6', '10'), ('10', '11'), ('10', '12'), ('10', '13'), ('10', '14'), ('14', '15'), ('15', '16'), ('15', '17'), ('14', '18'), ('18', '19'), ('18', '20'), ('10', '21'), ('21', '22'), ('22', '23'), ('22', '24'), ('21', '25'), ('21', '26'), ('26', '27'), ('26', '28'), ('26', '29'), ('21', '30'), ('30', '31'), ('30', '32'), ('32', '33'), ('21', '34'), ('34', '35'), ('34', '36'), ('21', '37'), ('37', '38'), ('37', '39'), ('39', '40'), ('37', '41'), ('41', '42'), ('21', '43'), ('43', '44'), ('43', '45'), ('43', '46'), ('21', '47'), ('6', '48'), ('48', '49'), ('48', '50'), ('48', '51'), ('48', '52'), ('52', '53'), ('53', '54'), ('53', '55'), ('52', '56'), ('56', '57'), ('56', '58'), ('48', '59'), ('59', '60'), ('6', '61'), ('61', '62'), ('61', '63'), ('61', '64'), ('61', '65'), ('65', '66'), ('66', '67'), ('66', '68'), ('65', '69'), ('69', '70'), ('69', '71'), ('61', '72'), ('72', '73'), ('73', '74

In [40]:
re_punctuation_string = '[()\s,/.\'_:]'
remove_token=['',' ',',',':','\n',';']

programs=[]
edges=[]
vocabulary = []
for f in files:
  lines = extract_source_code(f)
  edge_list= extract_ast_edges(f)
  edges.append(edge_list)
  program=[]
  for line in lines:
    statement=[token for token in re.split(re_punctuation_string,line)if token not in remove_token]
    program.append(statement)
  programs.append(program[2:])
  for s in program[2:]:
        for t in s:
           if t not in vocabulary:
             vocabulary.append(t) 
print(programs[1])
print(edges[0])
print(len(programs))
print(len(edges))
print(len(vocabulary))

[['IMPORTS'], ['IMPORT', 'testcasesupport'], ['IMPORT', 'javax', 'servlet', 'http'], ['CLASS'], ['MODIFIER', 'public'], ['NAME', 'CWE78', 'OS', 'Command', 'Injection', 'getCookies', 'Servlet', '01'], ['EXTENDS', 'AbstractTestCaseServlet'], ['METHOD'], ['MODIFIER', 'public'], ['RETURN', 'void'], ['NAME', 'bad'], ['PARAMS'], ['VAR'], ['TYPE', 'HttpServletRequest'], ['NAME', 'request'], ['VAR'], ['TYPE', 'HttpServletResponse'], ['NAME', 'response'], ['BLOCK'], ['VAR'], ['TYPE', 'String'], ['NAME', 'data'], ['30', 'data'], ['VAR'], ['TYPE', 'Cookie'], ['NAME', 'cookieSources'], ['INIT', '=', 'request', 'getCookies'], ['IF'], ['COND', 'cookieSources', '!=', 'null'], ['THEN'], ['38', 'data', '=', 'cookieSources[0]', 'getValue'], ['VAR'], ['TYPE', 'String'], ['NAME', 'osCommand'], ['IF'], ['COND', 'System', 'getProperty', 'os', 'name', 'toLowerCase', 'indexOf', 'win', '>=', '0'], ['THEN'], ['46', 'osCommand', '=', 'c', '\\\\\\\\WINDOWS\\\\\\\\SYSTEM32\\\\\\\\cmd', 'exe', 'c', 'dir'], ['ELSE']

In [41]:
# shuffle data
import random
randnum = random.randint(0,100)
random.seed(randnum)
random.shuffle(programs)

random.seed(randnum)
random.shuffle(labels)

random.seed(randnum)
random.shuffle(edges)

training_data = []
for i in range(1600):
  #training_data.append((programs[i],edges[i],labels[i]))
  training_data.append((programs[i],labels[i]))
print(training_data[0])
eval_data=[]
for i in range(1600,2100):
  #eval_data.append((programs[i],edges[i],labels[i]))
  eval_data.append((programs[i],labels[i]))
print(eval_data[0])

([['IMPORTS'], ['IMPORT', 'testcasesupport'], ['IMPORT', 'javax', 'servlet', 'http'], ['CLASS'], ['MODIFIER', 'public'], ['NAME', 'CWE89', 'SQL', 'Injection', 'Environment', 'executeBatch', '52b'], ['METHOD'], ['MODIFIER', 'public'], ['RETURN', 'void'], ['NAME', 'badSink'], ['PARAMS'], ['VAR'], ['TYPE', 'String'], ['NAME', 'data'], ['BLOCK'], ['27', 'new', 'CWE89', 'SQL', 'Injection', 'Environment', 'executeBatch', '52c', 'badSink', 'data'], ['METHOD'], ['MODIFIER', 'public'], ['RETURN', 'void'], ['NAME', 'goodG2BSink'], ['PARAMS'], ['VAR'], ['TYPE', 'String'], ['NAME', 'data'], ['BLOCK'], ['33', 'new', 'CWE89', 'SQL', 'Injection', 'Environment', 'executeBatch', '52c', 'goodG2BSink', 'data'], ['METHOD'], ['MODIFIER', 'public'], ['RETURN', 'void'], ['NAME', 'goodB2GSink'], ['PARAMS'], ['VAR'], ['TYPE', 'String'], ['NAME', 'data'], ['BLOCK'], ['39', 'new', 'CWE89', 'SQL', 'Injection', 'Environment', 'executeBatch', '52c', 'goodB2GSink', 'data']], 3)
([['IMPORTS'], ['IMPORT', 'testcasesup

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#look up table  
#using one-hot embedding
vocabulary_size = len(vocabulary)
def look_up_table(word_idx):
  x = torch.zeros(vocabulary_size).long()
  x[word_idx] = 1.0
  return x
word2idx = {w: idx +1 for (idx, w) in enumerate(vocabulary)}
idx2word = {idx+1: w for (idx, w) in enumerate(vocabulary)}
print(look_up_table(word2idx['!=']))

tensor([0, 0, 0,  ..., 0, 0, 0])


In [62]:

context_size = 100

def get_childs(id,edge):
  #print(edge)
  #print(id)
  childs=[]
  for (str_id,str_child) in edge:
    str2int_id= int(str_id)-3
    str2int_child = int(str_child)-3
    if id == str2int_id:
      if str2int_child<context_size:
        childs.append(str2int_child)
  return childs

alpha= 0.5
def ast_mordified(contexts,id,program,edge):
  childs = get_childs(id,edge) #[0,1,2,..]
  
  contexts_childs =torch.zeros(1).long()
  for child in childs:
    p = program[child]
    num = 0
    contexts_child =torch.zeros(1).long()
    for t in p:
      contexts_child=contexts_child+word2idx[t]
      num=num+1
    contexts_childs=contexts_childs+ (contexts_child/num)
  if len(childs)>0:
    result=contexts+((contexts_childs)/len(childs))*alpha
  else:
    result = contexts
  return result



def get_ast_context(program,edge):
  id = 0
  contexts =torch.zeros(context_size,1).long()

  for p in program:
    if id < context_size:
      num = 0
      for t in p:
        contexts[id]=contexts[id]+word2idx[t]
        num = num + 1
      contexts[id] = contexts[id]/num # average
      contexts[id] = ast_mordified(contexts[id],id,program,edge)
    id = id + 1
  return contexts
def get_source_context(program):
  id = 0
  contexts =torch.zeros(context_size,1).long()

  for p in program:
    if id < context_size:
      num = 0
      for t in p:
        contexts[id]=contexts[id]+word2idx[t]
        num = num + 1
      contexts[id] = contexts[id]/num # average
    id = id + 1
  return contexts

print(get_source_context(programs[0]))

tensor([[  1],
        [  2],
        [  4],
        [ 18],
        [ 19],
        [584],
        [ 31],
        [ 19],
        [ 60],
        [122],
        [129],
        [ 37],
        [ 36],
        [ 30],
        [ 36],
        [460],
        [ 31],
        [ 19],
        [ 60],
        [124],
        [129],
        [ 37],
        [ 36],
        [ 30],
        [ 36],
        [461],
        [ 31],
        [ 19],
        [ 60],
        [777],
        [129],
        [ 37],
        [ 36],
        [ 30],
        [ 36],
        [596],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [ 

In [0]:

torch.manual_seed(1)
embedding_dim =5

class FFNN(nn.Module):
  def __init__(self, vocab_size, embedding_dim):
    super(FFNN, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(embedding_dim*context_size,3)
  def forward(self, inputs):
    #print(inputs.size())
    embeds = self.embeddings(inputs.view(-1))
    #print(embeds)
    out = F.relu(self.linear1(embeds.view(-1)))
    out = F.softmax(out,dim=0)
    return out



In [64]:

loss_function = nn.BCELoss()  
model_ast = FFNN(vocabulary_size*2, embedding_dim)
optimizer_ffnn = optim.Adam(model_ast.parameters(), lr=0.001)
params = sum(p.numel() for p in model_ast.parameters() if p.requires_grad)
print("Total number of parameters for FFNN is: {}".format(params))

loss_ffnn=[]
for epoch in range(10):
  losses = 0
  pid=0
  #for program,edge,label in training_data:
  for program,label in training_data:  
    if label ==1:
      out_labels = torch.FloatTensor([1,0,0])
    elif label == 2:
      out_labels  = torch.FloatTensor([0,1,0])
    elif label == 3:
      out_labels  = torch.FloatTensor([0,0,1])

    context_idx = get_ast_context(program,edges[pid])
    pid = pid +1
    #context_idx = get_source_context(program)
    model_ast.zero_grad()
    #print(context_idx.size())
    out = model_ast(context_idx)
    #print(out.size())
    #print(f'Out {out},lables{out_labels}')
    #print(out_labels)
    loss = loss_function(out, out_labels)
    #print(f'Loss {loss.item()}')
    loss.backward()
    optimizer_ffnn.step()
    #print(f'Loss {loss.item()}')
    losses +=loss.item()
  print(f'Loss at epo {epoch}: {losses/len(training_data)}')
  loss_ffnn.append(losses/len(training_data))

Total number of parameters for FFNN is: 17893
Loss at epo 0: 0.3434656951712532
Loss at epo 1: 0.14382246189130454
Loss at epo 2: 0.09942753168340118
Loss at epo 3: 0.08041423795701476
Loss at epo 4: 0.06763117849235346
Loss at epo 5: 0.05938735683679822
Loss at epo 6: 0.05176512757000468
Loss at epo 7: 0.04733782216478966
Loss at epo 8: 0.04619986878986502
Loss at epo 9: 0.04186711548512701


In [66]:
with torch.no_grad():
  accu=0
  pid=1600
  for program, label in eval_data:
    context_idx = get_ast_context(program,edges[pid])
    pid = pid+1
    model_ast.zero_grad()
    out = model_ast(context_idx)
    #print(f'{out}:{label}')
    if(out[0]>(1/3)):
      if(label == 1):
        accu = accu +1
    elif(out[1]>(1/3)):
      if(label == 2):
        accu = accu +1
    elif(out[2]>(1/3)):
      if(label == 3):
        accu = accu +1

  print(f'Accuracy_source_code: {accu/(len(eval_data))}') 
#  print(f'Precision_ffnn: {pre}')
#  print(f'Recall_ffnn: {re}')
#  print(f'F1_ffnn: {2*(pre*re/(pre+re))}') 

Accuracy_source_code: 0.898
